```bash
before go through this notebook, please start CLIP server by using 
DOCKER_BUILDKIT=1 docker-compose -f services/docker-compose.svc.yml --env-file .env up -d 
```

In [2]:
import requests
import os
from datetime import datetime
from dotenv import load_dotenv
from pathlib import Path
import numpy as np
import pandas as pd 
from PIL import Image 
dotenv_path = Path('../.env.dev')
load_dotenv(dotenv_path=dotenv_path)

CLIP_PORT = os.environ.get("CLIP_PORT", None)
DIV_NOISE_PORT = os.environ.get("DIV_NOISE_PORT", None)

assert CLIP_PORT is not None, "CLIP_PORT is not set"
assert DIV_NOISE_PORT is not None, "DIV_NOISE_PORT is not set"

In [23]:
import pandas as pd 
from tqdm import tqdm 
df = pd.read_csv('../data/fact_30k.csv')
df.head(1)

,url,claim,content,top_image,index
0,https://leadstories.com/hoax-alert/2022/07/fac...,NASA used images of Devon Island to fake photo...,Analog Mission Did NASA use the landscape of a...,https://leadstories.com/caption_3479233.jpg,https://leadstories.com/hoax-alert/2022/07/fac...


There are 3 way to extract feature, I recommend using method 2 to keep to origin information of image.
But method 3 is more safety because u could test the image before send it to server. This will modify some pixels of image

method 1

In [4]:
def encode_image_url(image_url):
    api = f"api/image/url"
    url = f"http://localhost:{CLIP_PORT}/{api}"
    response = requests.post(url, json={"url": image_url})
    response = response.json() 
    return response['feature']

print(encode_image_url('https://leadstories.com/caption_3479233.jpg'))

{'data': [-0.02685546875, -0.050048828125, -0.0034427642822265625, 0.0233612060546875, -0.0193939208984375, 0.0049285888671875, -0.03204345703125, 0.052093505859375, -0.016387939453125, 0.019775390625, 0.041656494140625, -0.051177978515625, 0.03173828125, -0.01477813720703125, 0.0025081634521484375, 0.03656005859375, -0.03558349609375, -0.0288543701171875, -0.01299285888671875, -0.040924072265625, -0.061279296875, -0.00917816162109375, 0.023956298828125, -0.01422882080078125, -0.026153564453125, 0.031036376953125, 0.00624847412109375, -0.01053619384765625, -0.005252838134765625, 0.0309295654296875, -0.01666259765625, -0.0295867919921875, 0.01214599609375, -0.03497314453125, 0.01959228515625, 0.0261077880859375, 0.01611328125, 0.007534027099609375, -0.07647705078125, 0.013336181640625, -0.0168304443359375, -0.0233306884765625, 0.013824462890625, -0.03668212890625, 0.01739501953125, 0.07244873046875, 0.057861328125, -0.01079559326171875, -0.0186920166015625, -0.01044464111328125, -0.0188

method 2

In [10]:
def _download_content(image_url):
    image_bytes = requests.get(image_url, stream=True).raw
    return image_bytes

def test_encode_single_image(image_bytes):
    api = f"api/image/"
    url = f"http://localhost:{CLIP_PORT}/{api}"
    response = requests.post(url=url, files=[('data', image_bytes)])
    response = response.json() 
    return response['feature']

image_bytes = _download_content('https://leadstories.com/caption_3479233.jpg')
print(test_encode_single_image(image_bytes))

{'data': [-0.02685546875, -0.050048828125, -0.0034427642822265625, 0.0233612060546875, -0.0193939208984375, 0.0049285888671875, -0.03204345703125, 0.052093505859375, -0.016387939453125, 0.019775390625, 0.041656494140625, -0.051177978515625, 0.03173828125, -0.01477813720703125, 0.0025081634521484375, 0.03656005859375, -0.03558349609375, -0.0288543701171875, -0.01299285888671875, -0.040924072265625, -0.061279296875, -0.00917816162109375, 0.023956298828125, -0.01422882080078125, -0.026153564453125, 0.031036376953125, 0.00624847412109375, -0.01053619384765625, -0.005252838134765625, 0.0309295654296875, -0.01666259765625, -0.0295867919921875, 0.01214599609375, -0.03497314453125, 0.01959228515625, 0.0261077880859375, 0.01611328125, 0.007534027099609375, -0.07647705078125, 0.013336181640625, -0.0168304443359375, -0.0233306884765625, 0.013824462890625, -0.03668212890625, 0.01739501953125, 0.07244873046875, 0.057861328125, -0.01079559326171875, -0.0186920166015625, -0.01044464111328125, -0.0188

method 3

In [12]:
def _download_image(image_url, output_file='tmp.jpg'):
    image_bytes = requests.get(image_url, stream=True).raw
    im = Image.open(image_bytes)
    im.save(output_file)
    im.close()

def test_encode_single_image(filepath='tmp.jpg'):
    api = f"api/image/"
    url = f"http://localhost:{CLIP_PORT}/{api}"
    response = requests.post(url=url, files=[('data', open(filepath, 'rb'))])
    response = response.json() 
    return response['feature']

_download_image('https://leadstories.com/caption_3479233.jpg', 'tmp.jpg')
print(test_encode_single_image('tmp.jpg'))

{'data': [-0.022705078125, -0.056243896484375, -0.01136016845703125, 0.0147552490234375, -0.02276611328125, 0.006534576416015625, -0.03125, 0.046844482421875, -0.0182342529296875, 0.0145263671875, 0.039093017578125, -0.053924560546875, 0.031341552734375, -0.01318359375, 0.005123138427734375, 0.032440185546875, -0.038299560546875, -0.0372314453125, -0.0135345458984375, -0.04437255859375, -0.058807373046875, -0.007480621337890625, 0.0213470458984375, -0.015594482421875, -0.024627685546875, 0.025665283203125, 0.003162384033203125, -0.004802703857421875, -0.003658294677734375, 0.02716064453125, -0.021881103515625, -0.029632568359375, 0.01218414306640625, -0.0374755859375, 0.0211181640625, 0.0290985107421875, 0.010406494140625, 0.00879669189453125, -0.08203125, 0.00931549072265625, -0.015594482421875, -0.019744873046875, 0.01102447509765625, -0.03759765625, 0.0166778564453125, 0.06817626953125, 0.056732177734375, -0.015106201171875, -0.0211944580078125, -0.01214599609375, -0.0185546875, -0.

Im using method 3

In [30]:
df = df[:3]

def _download_image(image_url, output_file='tmp.jpg'):
    image_bytes = requests.get(image_url, stream=True).raw
    im = Image.open(image_bytes)
    im.save(output_file)
    im.close()


def test_encode_single_image(filepath='tmp.jpg'):
    api = f"api/image/"
    url = f"http://localhost:{CLIP_PORT}/{api}"
    response = requests.post(url=url, files=[('data', open(filepath, 'rb'))])
    response = response.json() 
    return response['feature']

In [35]:
embs = []
from io import BytesIO
for i, row in tqdm(df.iterrows(), total=len(df)):
    image_url = row['top_image']
    try:
        _download_image(image_url, 'tmp.jpg')
    except:
        print('image error', image_url)
        embs.append(None)
        continue
    emb = test_encode_single_image('tmp.jpg')['data']
    embs.append(emb)

100%|██████████| 3/3 [00:01<00:00,  2.13it/s]

image error https://factly.in/wp-content/uploads//2022/10/shepherd-girl-thumbnail.jpg


In [36]:
df['embeddings'] = embs
df.head()

,url,claim,content,top_image,index,embeddings
0,https://leadstories.com/hoax-alert/2022/07/fac...,NASA used images of Devon Island to fake photo...,Analog Mission Did NASA use the landscape of a...,https://leadstories.com/caption_3479233.jpg,https://leadstories.com/hoax-alert/2022/07/fac...,"[-0.022705078125, -0.056243896484375, -0.01136..."
1,https://www.boomlive.in/fact-check/no-this-is-...,Photo shows image of death certificate with PM...,An image of a provisional certificate of COVID...,https://www.boomlive.in/h-upload/2021/04/22/94...,https://www.boomlive.in/fact-check/no-this-is-...,"[0.009429931640625, 0.006649017333984375, -0.0..."
2,https://factly.in/the-moroccan-child-in-this-i...,Photo collage featuring the childhood and curr...,A collage containing the images of a girl chil...,https://factly.in/wp-content/uploads//2022/10/...,https://factly.in/the-moroccan-child-in-this-i...,None


In [37]:
df.to_csv('../data/fact_embs.csv', index=False)